In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
import folium
from pandas import json_normalize

In [3]:
geo=Nominatim(user_agent='hkjx')

CLIENT_ID="2Y10RMHFDSVKCQNL0FJV5L2LHRFFNV022NAGWHSC5FSY20L4"
CLIENT_SECRET="BGYY1LU0KEVTCCU23FIWRJHIHJWWMXDZPEZRLKPGNAFBD3RY"
VERSION="20200101"

In [149]:
source=requests.get('https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi').text
soup=BeautifulSoup(source,'lxml')
soup=soup.find('div',class_='mw-parser-output')

table=[]
district=[]
for i in soup.find_all('li'):
    table.append(i.text)
    
delhi=pd.DataFrame(table[12:198],columns=['Area'])

In [150]:
delhi['lat']=np.zeros(delhi.shape[0])
delhi['long']=np.zeros(delhi.shape[0])

In [151]:
delhi

,Area,lat,long
0,Adarsh Nagar,0.0,0.0
1,Ashok Vihar,0.0,0.0
2,Begum Pur,0.0,0.0
3,Karala,0.0,0.0
4,Model Town,0.0,0.0
...,...,...,...
181,Mayur Vihar,0.0,0.0
182,Meera Bagh,0.0,0.0
183,Mehrauli,0.0,0.0
184,Model Town,0.0,0.0


In [154]:
for i,area in delhi.iterrows():
    try: 
        address=(area['Neighborhood']+',Delhi')
        print(i,address)
        loc=geo.geocode(address)
        delhi.iloc[i,1]=loc.latitude
        delhi.iloc[i,2]=loc.longitude
    except:
        pass

0 Adarsh Nagar,Delhi
1 Ashok Vihar,Delhi
2 Azadpur,Delhi
3 Bawana,Delhi
4 Dhaka,Delhi
5 Jahangirpuri,Delhi
6 Karala,Delhi
7 Keshav Puram,Delhi
8 Kingsway Camp,Delhi
9 Kohat Enclave,Delhi
10 Model Town,Delhi
11 Narela,Delhi
12 Pitam Pura,Delhi
13 Rani Bagh,Delhi
14 Rithala,Delhi
15 Shalimar Bagh,Delhi
16 Shakti Nagar,Delhi
17 Bara Hindu Rao,Delhi
18 Chandni Chowk,Delhi
19 Chawri Bazaar,Delhi
20 Civil Lines,Delhi
21 Dariba Kalan,Delhi
22 Gulabi Bagh,Delhi
23 Kamla Nagar,Delhi
24 Kashmiri Gate,Delhi
25 Kotwali,Delhi
26 Lahori Gate,Delhi
27 Nehru Vihar,Delhi
28 Paharganj,Delhi
29 Pratap Nagar,Delhi
30 Pul Bangash,Delhi
31 Roshanara Bagh,Delhi
32 Sadar Bazaar,Delhi
33 Sangam Vihar,Delhi
34 Sant Nagar,Delhi
35 Sarai Kale Khan,Delhi
36 Sarai Rohilla,Delhi
37 Shakti Nagar,Delhi
38 Shastri Nagar,Delhi
39 Tis Hazari,Delhi
40 Timarpur,Delhi
41 Babarpur,Delhi
42 Dilshad Garden,Delhi
43 Kabir Nagar,Delhi
44 Mandoli,Delhi
45 Maujpur,Delhi
46 Mukherjee Nagar,Delhi
47 Nand Nagri,Delhi
48 Naveen Shahda

In [143]:
delhi=delhi[delhi['lat']!=0]

In [144]:
delhi.reset_index(drop=True,inplace=True)

In [163]:
loc=geo.geocode('Delhi')

In [164]:
delhi_map=folium.Map([loc.latitude,loc.longitude],zoom_start=10)
for lat,long,area in zip(delhi['lat'],delhi['long'],delhi['Area']):
    label=folium.Popup(area,parse_html=True)
    folium.features.CircleMarker([lat,long],popup=label,color='blue',radius=5,fill=True).add_to(delhi_map)
delhi_map

In [165]:
delhi.shape

(170, 5)

In [41]:
delhi.to_csv('delhi.csv')

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        print('.',end='')
        try:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                100)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal code', 
                  'Postal code Latitude', 
                  'Postal code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
delhi_venues=getNearbyVenues(delhi['Neighborhood'],delhi['lat'],delhi['long'])

..............................................................................................................................................................................

In [28]:
#delhi_venues.to_csv('dehi_venues.csv')
delhi_venues.head()

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,malwa,28.701699,77.185399,Farm
2,Ashok Vihar,28.699453,77.184826,Grand Plaza,28.696577,77.185245,Hotel Bar
3,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service
4,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery


In [29]:
delhi_venues.shape

(1339, 7)

In [98]:
venues=delhi_venues['Area'].unique()

In [135]:
delhi['Neighborhood'] in venues

C:\Users\dell\anaconda3-\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


ValueError: Lengths must match to compare

In [131]:
delhi_venues.groupby('Area').count()['Venue']>5


Area
Adarsh Nagar          False
Alaknanda              True
Anand Vihar           False
Ashok Nagar            True
Ashok Vihar            True
                      ...  
Vasundhara Enclave    False
Vikaspuri             False
Vishwas Nagar         False
Vivek Vihar           False
Wazirabad             False
Name: Venue, Length: 131, dtype: bool

In [51]:
unique_cat=delhi_venues['Venue Category'].unique()
restaurant_type=[]
for cat in unique_cat:
    if('Restaurant' in cat):
        restaurant_type.append(cat)
res_table=delhi_venues[delhi_venues['Venue Category'].isin(restaurant_type)]

In [52]:
res_table.reset_index(drop=True, inplace=True)
res_table

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant
2,Model Town,28.702714,77.193991,McDonald's,28.705639,77.190148,Fast Food Restaurant
3,Pitam Pura,28.703268,77.132250,Angrezee Dhaba,28.702592,77.133812,Indian Restaurant
4,Civil Lines,28.676851,77.225030,moets,28.678015,77.224739,Chinese Restaurant
...,...,...,...,...,...,...,...
368,Mehrauli,28.521826,77.178323,Oberoi Dhaba,28.521264,77.179785,Restaurant
369,Mehrauli,28.521826,77.178323,Tonino Ristorante,28.521796,77.177582,Restaurant
370,Mehrauli,28.521826,77.178323,THAI HIGH at the Qutab,28.522637,77.182351,Thai Restaurant
371,Model Town,28.702714,77.193991,McDonald's,28.705639,77.190148,Fast Food Restaurant


In [53]:
len(unique_cat)

175

In [172]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add area column back to dataframe
delhi_onehot['Area'] = delhi_venues['Area'] 

# move area column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_grouped=delhi_onehot.groupby('Area').mean().reset_index()
delhi_grouped.head()

,Area,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Arcade,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Travel Lounge,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [173]:
from sklearn.cluster import KMeans 
# set number of clusters
kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)

131

In [174]:
delhi_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [175]:
delhi_grouped

,Cluster Labels,Area,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Travel Lounge,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio
0,1,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Ashok Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,Ashok Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,0,Vasundhara Enclave,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,1,Vikaspuri,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128,0,Vishwas Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
129,2,Vivek Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
